In [ ]:
# !pip install "pymilvus[model,milvus_lite]" -q

In [1]:
!pip install pymilvus[model]==2.6.6 milvus-lite==2.5.1 pypdf==6.4.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pypdf import PdfReader
from typing import List
import os

# ------------------- Embedding Model -------------------
print("Loading embedding model: Alibaba-NLP/gte-multilingual-base ...")

embeding_model = 'all-MiniLM-L6-v2'
# embeding_model = 'Alibaba-NLP/gte-multilingual-base'

embedding_model = SentenceTransformer(
    embeding_model,
    trust_remote_code=True   # ← This fixes the ValueError
)
DIM = embedding_model.get_sentence_embedding_dimension()  # Should be 768

def embed_fn(texts: List[str], is_query: bool = False) -> List[List[float]]:
    """Unified embedding function for documents and queries"""
    # gte models work significantly better with this instruction prefix for queries
    if is_query:
        prefixed = [f"Represent this sentence for searching relevant passages: {t}" for t in texts]
    else:
        prefixed = texts

    embeddings = embedding_model.encode(
        prefixed,
        normalize_embeddings=True,      # Very important for cosine similarity
        batch_size=32,
        show_progress_bar=False,
        convert_to_numpy=True
    )
    return embeddings.tolist()

# ------------------- LLM -------------------
# Currently one of the best reasonable local instruction-tuned models (2025/2026)
# model_id = "Qwen/Qwen3-1.7B"           # ← most recommended choice
model_id = "google/gemma-3-4b-it"
# Alternatives:
# model_id = "Qwen/Qwen2.5-7B-Instruct"
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "mistralai/Mistral-Nemo-Instruct-2407"

print(f"Loading LLM: {model_id} (this may take several minutes)...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# ------------------- Milvus Lite -------------------
DB_FILE = "pdf_rag_local_gte.db"           # changed name to avoid conflict with old db
client = MilvusClient(DB_FILE)
collection_name = "pdf_collection"

# ------------------- Helpers -------------------
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        reader = PdfReader(pdf_path)
        text = "\n".join(page.extract_text() or "" for page in reader.pages)
        return text.strip()
    except Exception as e:
        return f"Error reading PDF: {e}"

def chunk_text(text: str, chunk_size: int = 1200, overlap: int = 300) -> List[str]:
    if not text.strip():
        return []
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + chunk_size, text_len)
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        if end >= text_len:
            break
        start = end - overlap
    return [c for c in chunks if len(c) > 80]

def init_collection():
    if client.has_collection(collection_name):
        client.drop_collection(collection_name)

    client.create_collection(
        collection_name=collection_name,
        dimension=DIM,
        metric_type="COSINE",
        auto_id=False,
        enable_dynamic_field=True
    )

    # Insert sample data
    sample_docs = [
        "Artificial intelligence was founded as an academic discipline in 1956.",
        "The Dartmouth Conference in 1956 is considered the founding event of AI.",
        "John McCarthy coined the term 'Artificial Intelligence' in 1955.",
        "Alan Turing conducted pioneering research in AI and computing.",
        "Machine learning is a subset of artificial intelligence focused on data-driven algorithms.",
    ]

    vectors = embed_fn(sample_docs, is_query=False)
    data = [
        {"id": i, "vector": v, "text": d, "source": "sample"}
        for i, (d, v) in enumerate(zip(sample_docs, vectors))
    ]
    client.insert(collection_name, data)

    # Create index
    index_params = client.prepare_index_params()
    index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="COSINE")
    client.create_index(collection_name, index_params)
    client.load_collection(collection_name)

# Safe way to get all existing IDs
def get_all_existing_ids(client, collection_name: str, batch_size: int = 8000) -> List[int]:
    all_ids = []
    offset = 0

    while True:
        try:
            res = client.query(
                collection_name=collection_name,
                filter="",
                output_fields=["id"],
                limit=batch_size,
                offset=offset
            )
            if not res:
                break

            batch_ids = [r["id"] for r in res]
            all_ids.extend(batch_ids)

            if len(res) < batch_size:
                break

            offset += batch_size

        except Exception as e:
            print(f"Error during ID pagination: {e}")
            break

    return all_ids

# Initialize collection if needed
if not client.has_collection(collection_name):
    init_collection()
else:
    client.load_collection(collection_name)

# Get next available ID
existing_ids = get_all_existing_ids(client, collection_name)
next_id = max(existing_ids, default=-1) + 1
print(f"Next available ID will start from: {next_id}")

# ------------------- PDF Indexing -------------------
def process_pdfs(pdf_files: List[gr.File], chunk_size: int, chunk_overlap: int, progress=gr.Progress()):
    global next_id
    if not pdf_files:
        return "No PDFs uploaded."

    # Remove previous user documents
    client.delete(collection_name, filter="source like 'pdf_%'")

    progress(0, desc="Starting...")
    total_chunks = 0
    messages = []

    for idx, file_obj in enumerate(pdf_files):
        filename = os.path.basename(file_obj.name)
        progress((idx + 0.1) / len(pdf_files), desc=f"Reading {filename}...")
        text = extract_text_from_pdf(file_obj.name)

        if text.startswith("Error"):
            messages.append(f"✗ Failed to read: {filename}")
            continue

        progress((idx + 0.4) / len(pdf_files), desc=f"Chunking {filename}...")
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=chunk_overlap)

        if not chunks:
            messages.append(f"✗ No useful text extracted from {filename}")
            continue

        progress((idx + 0.7) / len(pdf_files), desc=f"Embedding {len(chunks)} chunks...")
        vectors = embed_fn(chunks, is_query=False)

        data = []
        for j, (chunk, vec) in enumerate(zip(chunks, vectors)):
            data.append({
                "id": next_id,
                "vector": vec,
                "text": chunk,
                "source": f"pdf_{filename}_chunk_{j}"
            })
            next_id += 1

        progress((idx + 0.9) / len(pdf_files), desc="Saving to database...")
        res = client.insert(collection_name, data)
        total_chunks += res["insert_count"]
        messages.append(f"✓ {len(chunks)} chunks from {filename}")

    return f"**Indexing complete!**\nTotal chunks: **{total_chunks}**\n\n" + "\n".join(messages)

# ------------------- RAG Query -------------------
def rag_query(question: str, top_k: int = 6, temperature: float = 0.75):
    if not question.strip():
        return "Please enter a question.", ""

    # Use query prefix for better retrieval
    query_vec = embed_fn([question], is_query=True)[0]

    results = client.search(
        collection_name=collection_name,
        data=[query_vec],
        limit=top_k,
        output_fields=["text", "source"],
        search_params={"metric_type": "COSINE"}
    )[0]

    context_lines = []
    for hit in results:
        score = hit["distance"]
        source = hit["entity"]["source"]
        text = hit["entity"]["text"]
        preview = text[:480] + "..." if len(text) > 480 else text
        context_lines.append(f"[{score:.3f}] {source}\n{preview}")

    context = "\n\n".join(context_lines)

    prompt = f"""You are a helpful, precise assistant. Answer the question based **only** on the provided context.
If the information is insufficient or not present, reply only with "I don't know."

Context:
{context}

Question: {question}

Answer:"""

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)
        outputs = llm.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=True,
            temperature=temperature,
            top_p=0.92,
            pad_token_id=tokenizer.eos_token_id
        )
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = full_output[len(prompt):].strip()
    except Exception as e:
        answer = f"Generation error: {str(e)}"

    context_display = "Retrieved context (score | source):\n\n" + context
    return answer, context_display

# ------------------- Gradio Interface -------------------
with gr.Blocks(title="PDF RAG • GTE-Multilingual", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📘 Local PDF RAG Chat\n**Embedding: gte-multilingual-base** • 100% offline")

    with gr.Tab("📤 Upload & Index"):
        pdf_input = gr.File(
            label="Upload PDF file(s)",
            file_count="multiple",
            file_types=[".pdf"],
            type="filepath"
        )
        with gr.Row():
            chunk_size = gr.Slider(400, 6000, value=1200, step=100, label="Chunk Size (characters)")
            chunk_overlap = gr.Slider(50, 1500, value=300, step=50, label="Chunk Overlap")
        index_button = gr.Button("🚀 Index PDFs", variant="primary", size="lg")
        status_output = gr.Textbox(label="Indexing Status", lines=10, interactive=False)

        index_button.click(
            process_pdfs,
            inputs=[pdf_input, chunk_size, chunk_overlap],
            outputs=status_output
        )

    with gr.Tab("❓ Ask Questions"):
        gr.Markdown("Ask anything about the indexed documents (supports multilingual questions)")
        question_input = gr.Textbox(
            label="Your question",
            placeholder="What are the main conclusions of the report? / 報告的主要結論是什麼？",
            lines=4
        )
        with gr.Row():
            top_k_slider = gr.Slider(2, 12, value=6, step=1, label="Top-K chunks")
            temperature_slider = gr.Slider(0.1, 1.5, value=0.75, step=0.05, label="Temperature")
        ask_button = gr.Button("Ask", variant="primary", size="lg")
        answer_output = gr.Textbox(label="Answer", lines=10)
        context_output = gr.Textbox(label="Retrieved Context", lines=12, interactive=False)

        ask_button.click(
            rag_query,
            inputs=[question_input, top_k_slider, temperature_slider],
            outputs=[answer_output, context_output]
        )

        gr.Examples(
            examples=[
                "When was artificial intelligence founded as an academic discipline?",
                "What did John McCarthy contribute to AI?",
                "報告的主要發現是什麼？",
                "Summarize the key findings from the uploaded document."
            ],
            inputs=question_input
        )

if __name__ == "__main__":
    print("Starting Gradio app on http://localhost:7860 (public link enabled)")
    demo.launch(
        server_name="0.0.0.0",
        server_port=7862,
        debug=False,
        share=True   # public temporary link
    )